In [1]:
from database.market import Market
from database.adatabase import ADatabase
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
market = Market()

In [3]:
financial_db = ADatabase("strategy_financial")

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
econ = market.retrieve("econ")
prices = market.retrieve("alpha_prices")
financials = market.retrieve("unified_financials")
market.disconnect()

In [5]:
econ

,date,year,quarter,month,adjclose,fedfunds,gdp,crude,cpi,unemployment
0,2000-02-01,2000.0,1.0,2.0,91.243927,5.676667,1.025235e+13,28.28,63.525534,4.1
1,2000-03-01,2000.0,1.0,3.0,99.833031,5.676667,1.025235e+13,31.71,63.525534,4.0
2,2000-05-01,2000.0,2.0,5.0,95.053322,6.273333,1.025235e+13,25.84,64.181506,4.0
3,2000-06-01,2000.0,2.0,6.0,96.696457,6.273333,1.025235e+13,30.19,64.181506,4.0
4,2000-08-01,2000.0,3.0,8.0,101.635895,6.520000,1.025235e+13,27.85,64.685434,4.1
...,...,...,...,...,...,...,...,...,...,...
1730,2020-12-24,2020.0,4.0,12.0,23735.949219,0.090000,2.095303e+13,48.18,96.350188,6.7
1731,2020-12-28,2020.0,4.0,12.0,27084.808594,0.090000,2.095303e+13,47.50,96.350188,6.7
1732,2020-12-29,2020.0,4.0,12.0,27362.437500,0.090000,2.095303e+13,47.85,96.350188,6.7
1733,2020-12-30,2020.0,4.0,12.0,28840.953125,0.090000,2.095303e+13,48.24,96.350188,6.7


In [ ]:
ticker = "PM"

In [6]:
prices = p.column_date_processing(prices)
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["year"] = [x.year for x in prices["date"]]

In [7]:
financials = p.column_date_processing(financials)
financials["quarter"] = [x.quarter for x in financials["date"]]
financials["year"] = [x.year for x in financials["date"]]

In [8]:
prices["close"] = [float(x) for x in prices["close"]]

In [9]:
labels = prices[["year","quarter","ticker","close"]].groupby(["year","quarter","ticker"]).mean().reset_index()
labels["year"] = labels["year"] - 3

In [10]:
data = financials.merge(labels,on=["year","quarter","ticker"]).merge(econ.drop("date",axis=1),on=["year","quarter"]).dropna()

In [12]:
data[data["ticker"]=="PM"].columns

,assets,liabilitiesandstockholdersequity,netcashprovidedbyusedinfinancingactivities,netcashprovidedbyusedininvestingactivities,date,ticker,quarter,year,close,month,adjclose,fedfunds,gdp,crude,cpi,unemployment
724,3.304950e+10,3.304950e+10,-2.815500e+09,-6.840000e+08,2009-08-14,PM,3,2009,90.410635,7.0,77.342995,0.156667,1.444893e+13,69.32,79.328674,9.5
725,3.304950e+10,3.304950e+10,-2.815500e+09,-6.840000e+08,2009-08-14,PM,3,2009,90.410635,9.0,82.650009,0.156667,1.444893e+13,68.11,79.328674,9.8
726,3.304950e+10,3.304950e+10,-2.815500e+09,-6.840000e+08,2009-08-14,PM,3,2009,90.410635,7.0,77.342995,0.156667,1.444893e+13,69.32,79.328674,9.5
727,3.304950e+10,3.304950e+10,-2.815500e+09,-6.840000e+08,2009-08-14,PM,3,2009,90.410635,9.0,82.650009,0.156667,1.444893e+13,68.11,79.328674,9.8
1792,3.376200e+10,3.376200e+10,-4.894333e+09,-2.281667e+09,2010-02-26,PM,1,2010,89.997167,2.0,87.562904,0.133333,1.499205e+13,74.41,80.611392,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576319,3.892150e+10,3.892150e+10,-2.444500e+09,-1.307000e+09,2019-04-25,PM,2,2019,99.844957,6.0,11011.102539,2.396667,2.143322e+13,57.73,94.924558,3.6
576320,3.892150e+10,3.892150e+10,-2.444500e+09,-1.307000e+09,2019-04-25,PM,2,2019,99.844957,6.0,11790.916992,2.396667,2.143322e+13,57.63,94.924558,3.6
576321,3.892150e+10,3.892150e+10,-2.444500e+09,-1.307000e+09,2019-04-25,PM,2,2019,99.844957,6.0,13016.231445,2.396667,2.143322e+13,59.17,94.924558,3.6
576322,3.892150e+10,3.892150e+10,-2.444500e+09,-1.307000e+09,2019-04-25,PM,2,2019,99.844957,6.0,11182.806641,2.396667,2.143322e+13,59.18,94.924558,3.6


In [60]:
start_year = 2016
end_year = 2022
factors = ["assets","liabilitiesandstockholdersequity"
           ,"netcashprovidedbyusedinfinancingactivities"
           ,"netcashprovidedbyusedininvestingactivities"]
performance = []
sim = []
financial_db.connect()
for year in tqdm(range(start_year,end_year)):
    try:
        training_start_date = datetime(year - 6,1,1)
        training_end_date = datetime(year,1,1)
        prediction_end_date = datetime(year+1,1,1)
        training_data = data[(data["date"]>=training_start_date) & (data["date"]<training_end_date)].reset_index(drop=True)
        if training_data.index.size > 100:
            prediction_data = data[(data["date"]>=training_end_date) & (data["date"]<prediction_end_date)]
            refined_data = {"X":training_data[factors],"y":training_data[["close"]].rename(columns={"close":"y"})}
            models = m.regression(refined_data)
            models["year"] = year
            for row in models.iterrows():
                api = row[1]["api"]
                prediction = row[1]["model"].predict(prediction_data[factors])
                prediction_data[f"{api}_prediction"] = prediction
                prediction_data[f"{api}_score"] = row[1]["score"]
            included_columns = ["year","quarter","ticker"]
            included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
            prediction_data["year"] = prediction_data["year"] + 3
            sim.append(prediction_data[included_columns])
#             financial_db.store("sim",prediction_data[included_columns])
    except Exception as e:
        print(year,str(e))
financial_db.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:50<00:00, 18.42s/it]

2021 Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.


In [61]:
# models["year"] = 2020
# for row in models.iterrows():
#     api = row[1]["api"]
#     prediction = row[1]["model"].predict(prediction_data[factors])
#     prediction_data[f"{api}_prediction"] = prediction
#     prediction_data[f"{api}_score"] = row[1]["score"]
# included_columns = ["year","quarter","ticker"]
# included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
# prediction_data["year"] = prediction_data["year"] + 1
# sim.append(prediction_data[included_columns])

In [62]:
# pd.concat(sim)

In [63]:
# start_year = 2022
# end_year = 2023
# factors = ["assets","liabilitiesandstockholdersequity"
#            ,"netcashprovidedbyusedinfinancingactivities"
#            ,"netcashprovidedbyusedininvestingactivities"]
# performance = []
# sim = []
# financial_db.connect()
# for year in tqdm(range(start_year,end_year)):
#     try:
#         training_start_date = datetime(year - 6,1,1)
#         training_end_date = datetime(year,1,1)
#         prediction_end_date = datetime(year+1,1,1)
#         training_data = data[(data["date"]>=training_start_date) & (data["date"]<training_end_date)].reset_index(drop=True)
#         if training_data.index.size > 100:
#             prediction_data = data[(data["date"]>=training_end_date) & (data["date"]<prediction_end_date)]
#             refined_data = {"X":training_data[factors],"y":training_data[["close"]].rename(columns={"close":"y"})}
#             models = m.regression(refined_data)
#             models["year"] = year
#             models["model"] = [pickle.dumps(x) for x in models["model"]]
#             models["year"] = year
#             financial_db.store("models",models)
#     except Exception as e:
#         print(year,str(e))
# financial_db.disconnect()

In [64]:
# start_year = 2021
# end_year = 2022
# factors = ["assets","liabilitiesandstockholdersequity"
#            ,"netcashprovidedbyusedinfinancingactivities"
#            ,"netcashprovidedbyusedininvestingactivities"]
# performance = []
# sim = []
# financial_db.connect()
# for year in tqdm(range(start_year,end_year)):
#     try:
#         training_end_date = datetime(year,1,1)
#         prediction_end_date = datetime(year+1,1,1)
#         prediction_data = financials[(financials["date"]>=training_end_date) & (financials["date"]<prediction_end_date)].dropna()
#         models = financial_db.retrieve("models")
#         models["model"] = [pickle.loads(x) for x in models["model"]]
#         for row in models.iterrows():
#             api = row[1]["api"]
#             prediction = row[1]["model"].predict(prediction_data[factors])
#             prediction_data[f"{api}_prediction"] = prediction
#             prediction_data[f"{api}_score"] = row[1]["score"]
#         included_columns = ["year","quarter","ticker"]
#         included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
#         prediction_data["year"] = prediction_data["year"] + 1
#         financial_db.store("predictions",prediction_data[included_columns])
#     except Exception as e:
#         print(year,str(e))
# financial_db.disconnect()

In [65]:
prediction_data

,assets,liabilitiesandstockholdersequity,netcashprovidedbyusedinfinancingactivities,netcashprovidedbyusedininvestingactivities,date,ticker,quarter,year,close,month,adjclose,fedfunds,gdp,crude,cpi,unemployment


In [66]:
simulation = pd.concat(sim).groupby(["year","quarter","ticker"]).mean().reset_index()
simulation = prices.merge(simulation,on=["year","quarter","ticker"]).dropna()

In [67]:
simulation["average_prediction"] = [sum([row[1][col] for col in simulation.columns if "prediction" in col]) for row in simulation.iterrows()]

In [68]:
simulation["average_prediction"] = simulation["average_prediction"] / 3
simulation["delta"] = (simulation["average_prediction"] - simulation["close"]) / simulation["close"]

In [69]:
simulation

,date,open,high,low,close,volume,ticker,quarter,year,skl_prediction,skl_score,xgb_prediction,xgb_score,light_prediction,light_score,cat_prediction,cat_score,average_prediction,delta
0,2019-01-02,187.8200,190.9900,186.7000,190.95,2475193,MMM,1,2019,75.192015,0.004208,128.787872,0.628398,75.412409,-1.719712,101.944557,0.040121,127.112284,-0.334316
1,2019-01-03,188.2800,188.2800,182.8900,183.76,3358241,MMM,1,2019,75.192015,0.004208,128.787872,0.628398,75.412409,-1.719712,101.944557,0.040121,127.112284,-0.308270
2,2019-01-04,186.7500,191.9800,186.0300,191.32,2995052,MMM,1,2019,75.192015,0.004208,128.787872,0.628398,75.412409,-1.719712,101.944557,0.040121,127.112284,-0.335604
3,2019-01-07,191.3600,192.3000,188.6600,190.88,2162158,MMM,1,2019,75.192015,0.004208,128.787872,0.628398,75.412409,-1.719712,101.944557,0.040121,127.112284,-0.334072
4,2019-01-08,193.0000,194.1100,189.5800,191.68,2479787,MMM,1,2019,75.192015,0.004208,128.787872,0.628398,75.412409,-1.719712,101.944557,0.040121,127.112284,-0.336852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680470,2022-04-05,116.8300,118.8700,116.6800,117.64,2001558,YUM,2,2022,104.736417,0.014576,220.118637,0.835178,101.529670,0.148731,124.879856,0.632154,183.754860,0.562010
680471,2022-04-06,117.0200,119.3700,116.0500,118.62,1683142,YUM,2,2022,104.736417,0.014576,220.118637,0.835178,101.529670,0.148731,124.879856,0.632154,183.754860,0.549105
680472,2022-04-07,118.2700,120.8350,117.4900,120.40,2008941,YUM,2,2022,104.736417,0.014576,220.118637,0.835178,101.529670,0.148731,124.879856,0.632154,183.754860,0.526203
680473,2022-04-08,118.7200,120.6500,118.7200,119.17,1743213,YUM,2,2022,104.736417,0.014576,220.118637,0.835178,101.529670,0.148731,124.879856,0.632154,183.754860,0.541956


In [70]:
positions = 10
start_date = simulation["date"].min()
end_date = simulation["date"].max()
reqs = [1,3,5,10,15]
limit = 14
values = [True,False]
trades = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            date = start_date
            iterration_sim = simulation.copy()
            if not value:
                iterration_sim["delta"] = iterration_sim["delta"] * -1
            while date < end_date:
                try:
                    todays_recs = iterration_sim[iterration_sim["date"]==date]
                    todays_recs.sort_values("delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0:
                        offering = todays_recs.iloc[position]
                        if offering["delta"] > req:
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["close"]
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:limit]
                            exits["gains"] = (exits["close"] - buy_price) / buy_price
                            exit = exits.sort_values("gains",ascending=False).iloc[0]
                            trade["sell_price"] = exit["close"]
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["value"] = value
                            trade["req"] = req
                            trade["position"] = position
                            trades.append(trade)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date+timedelta(days=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [08:29<00:00, 254.83s/it]


In [71]:
t = pd.DataFrame(trades)

In [72]:
t["delta"] =  (t["sell_price"] - t["close"]) / t["close"]

In [73]:
analysis = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            position_trades = t[(t["value"]==value) & (t["req"]==req) & (t["position"]==position)].sort_values("date")
            position_trades["delta"] = (position_trades["sell_price"] - position_trades["close"]) / position_trades["close"]
            initial = 100 / positions
            for delta in position_trades["delta"]:
                initial = initial * (1+delta)
            analysis.append({
                "value":value,
                "req":req,
                "position":position,
                "pv":initial
            })
            

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 89.26it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 17.39it/s]


In [74]:
complete = pd.DataFrame(analysis).pivot_table(index=["value","req"],columns="position",values="pv").reset_index()

In [75]:
complete["pv"] = [sum([row[1][i] for i in range(10)]) for row in complete.iterrows()]

In [76]:
complete.sort_values("pv",ascending=False)

position,value,req,0,1,2,3,4,5,6,7,8,9,pv
5,True,1,753.770501,838.609857,530.196850,2918.263325,967.176305,905.417502,4713.342084,2376.694937,540.616190,923.915376,15468.002926
6,True,3,753.770501,838.609857,530.196850,2918.263325,967.176305,905.417502,4713.342084,2376.694937,540.616190,923.915376,15468.002926
7,True,5,753.770501,838.609857,530.196850,2811.791925,912.483955,904.292803,3830.081153,1073.091087,209.144197,232.393891,12095.856219
8,True,10,252.834190,102.476641,50.420141,127.001946,48.152543,43.501318,60.823288,30.749993,18.117692,29.230911,763.308663
9,True,15,76.354265,84.284478,14.974046,13.871960,18.735771,15.710052,34.878881,12.329854,11.664298,11.664298,294.467903
0,False,1,10.894817,10.055642,10.055642,10.055642,10.055642,10.044343,10.000000,10.000000,10.000000,10.000000,101.161728
1,False,3,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,100.000000
2,False,5,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,100.000000
3,False,10,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,100.000000
4,False,15,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,100.000000


In [77]:
t[(t["value"]==True) & (t["req"]==1)]

,date,open,high,low,close,volume,ticker,quarter,year,skl_prediction,...,light_score,cat_prediction,cat_score,average_prediction,delta,sell_price,sell_date,value,req,position
236118,2019-01-02,11.7800,12.1600,11.7200,12.14,10385799,HBAN,1,2019,74.836490,...,-1.719712,78.595887,0.040121,114.901253,0.119440,13.59,2019-01-18,True,1,0
236131,2019-01-22,13.5100,13.6300,13.4500,13.52,14975032,HBAN,1,2019,74.836490,...,-1.719712,78.595887,0.040121,114.901253,0.032544,13.96,2019-02-07,True,1,0
236144,2019-02-08,13.8700,13.9200,13.5300,13.62,10787820,HBAN,1,2019,74.836490,...,-1.719712,78.595887,0.040121,114.901253,0.058003,14.41,2019-02-28,True,1,0
236158,2019-03-01,14.4500,14.6100,14.3200,14.40,12398531,HBAN,1,2019,74.836490,...,-1.719712,78.595887,0.040121,114.901253,-0.002083,14.37,2019-03-04,True,1,0
236160,2019-03-05,14.3500,14.3800,14.0500,14.27,10636720,HBAN,1,2019,74.836490,...,-1.719712,78.595887,0.040121,114.901253,-0.014015,14.07,2019-03-14,True,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602732,2022-03-21,11.1000,11.2500,10.9200,10.96,9504232,LUMN,1,2022,130.761152,...,0.148731,174.085126,0.632154,236.833084,0.040146,11.40,2022-03-25,True,1,9
602737,2022-03-28,11.1900,11.2100,10.9400,11.21,10243682,LUMN,1,2022,130.761152,...,0.148731,174.085126,0.632154,236.833084,0.015165,11.38,2022-03-30,True,1,9
603049,2022-03-31,11.3800,11.4500,11.2400,11.27,9076825,LUMN,1,2022,130.761152,...,0.148731,174.085126,0.632154,236.833084,0.023070,11.53,2022-04-04,True,1,9
679669,2022-04-05,15.3900,15.6200,15.0300,15.18,2824368,UA,2,2022,103.947588,...,0.148731,102.857752,0.632154,135.705911,-0.019763,14.88,2022-04-06,True,1,9


In [78]:
financial_db.connect()
financial_db.store("trades",t)
financial_db.disconnect()